# Orientation Estimation with Quaternions

## Estimation using Angular Rate

Let the instantaneous output of a 3-axis gyroscope, measured in degrees per second, be represented as $\boldsymbol{\omega}^{(body)} =\boldsymbol{\omega} = (0\ ,\ \omega_{x}\ ,\ \omega_{y}\ ,\ \omega_{z})$.
The orientation of the world frame relative to the body frame at time $t$ is expressed in quaternions as $\mathbf{q}_{t}^{(world)} = \mathbf{q}_{t} = (q_{0}\ ,\ q_{1}\ ,\ q_{2}\ ,\ q_{3})$ and the initial orientation is defined as $\mathbf{q}_{0} = (1\ ,\ 0\ ,\ 0\ ,\ 0)$.

The orientation estimate at time $t + \Delta t $ is $ \mathbf{q}_{t + \Delta t}$ .

From Taylor series we have, $f(x + h) = f(x) + h f'(x) + O(h^{2})$. Here, $O(h^{2})$ represents the order of magnitude of the error term.

$$\mathbf{q}_{t + \Delta t} = \mathbf{q}_{t} + \Delta t \ \dot{\mathbf{q}}_{t} \tag{1}$$

From Quartenion differentiation,

$$\begin{split}\begin{array}{rcl}
\mathbf{q}_{t + \Delta t} = \mathbf{q}_{t} + \Delta t \ \frac{1}{2} \ \boldsymbol{\omega} \ \mathbf{q}_{t} \\
= \mathbf{q}_{t} + \frac{1}{2} \ \boldsymbol{\omega} \ \mathbf{q}_{t} \ \Delta t
\end{array}\end{split} \tag{2}$$

After Quartenion multiplication of $\boldsymbol{\omega}$ and $\mathbf{q}_{t}$ ,

$$\mathbf{q}_{t + \Delta t} =
\begin{bmatrix}
    q_{0} \\
    q_{1} \\
    q_{2} \\
    q_{3}
\end{bmatrix}
+ \frac{1}{2} 
\begin{bmatrix}
    - q_{1} \ \omega_{x} - q_{2} \ \omega_{y} - q_{3} \ \omega_{z} \\
    q_{0} \ \omega_{x} + q_{2} \ \omega_{z} - q_{3} \ \omega_{y} \\
    q_{0} \ \omega_{y} - q_{1} \ \omega_{z} + q_{3} \ \omega_{x} \\
    q_{0} \ \omega_{z} + q_{1} \ \omega_{y} - q_{2} \ \omega_{x}
\end{bmatrix}
\Delta t \tag{3}$$

Normalize the result to get unit vector.

$$\mathbf{q}_{t + \Delta t} = \frac{\mathbf{q}_{t + \Delta t}}{\| \mathbf{q}_{t + \Delta t} \|} \tag{4}$$

- The implementation can be observed in [mpu9250 arduino library](https://github.com/hideakitai/MPU9250/blob/3741120ae5816aa0cb7d4e15870f99c875c72bc1/MPU9250/QuaternionFilter.h#L50).

## Estimation using Madgwick Filter

The Madgwick filter functions as a complementary filter that combines high-pass filtered gyroscope measurements with low-pass filtered measurements from accelerometer and magnetometer.

### Gradient Descent Algorithm

- Given Objective/Cost function,

    $$\begin{split}\begin{array}{rcl}
    f(\mathbf{q}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)}) &=& \mathbf{q}^{*} \ \mathbf{d}^{(world)} \ \mathbf{q}-\mathbf{s}^{(body)} \\
    &=&\begin{bmatrix}
    2d_x(\frac{1}{2}-q_y^2-q_z^2) + 2d_y(q_wq_z+q_xq_y) + 2d_z(q_xq_z-q_wq_y) - s_x \\
    2d_x(q_xq_y-q_wq_z) + 2d_y(\frac{1}{2}-q_x^2-q_z^2) + 2d_z(q_wq_x+q_yq_z) - s_y \\
    2d_x(q_wq_y+q_xq_z) + 2d_y(q_yq_z-q_wq_x) + 2d_z(\frac{1}{2}-q_x^2-q_y^2) - s_z \\
    \end{bmatrix}
    \end{array}\end{split} \tag{5}$$

    where,

    $\mathbf{q}$ is the orientation of the sensor, defined as $\mathbf{q}=(q_{w}\ ,\ q_{x}\ ,\ q_{y}\ ,\ q_{z})$ and $\mathbf{q}^{*}$ is conjugate of $\mathbf{q}$.

    $\mathbf{d}^{(world)}$ is the predefined reference in the world frame, defined as $\mathbf{d}^{(world)}=(0\ ,\ d_{x}\ ,\ d_{y}\ ,\ d_{z})$.

    $\mathbf{s}^{(body)}$ is the measured direction in the sensor frame, defined as $\mathbf{s}^{(body)}=(0\ ,\ s_{x}\ ,\ s_{y}\ ,\ s_{z})$.

- The solution of $\mathbf{q}$ can be obtained by minimizing $f(\mathbf{q}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})$.

- The gradient descent algorithm is

    $$\mathbf{q}_{k+1} = \mathbf{q}_{k} - \mu \frac{\nabla f(\mathbf{q}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})}{\|\nabla f(\mathbf{q}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})\|} \tag{6}$$

    where,
    
    $\mathbf{\mu}$ is the step-size and $k=0,1,2\dots n$ .

    $$\begin{split}\begin{array}{rcl}
    \nabla f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})
    &=& J(\mathbf{q}_{k}) \cdot f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)}) \\
    &=& \begin{bmatrix}
    \frac{\partial f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})}{\partial q_w} \\
    \frac{\partial f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})}{\partial q_x} \\
    \frac{\partial f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})}{\partial q_y} \\
    \frac{\partial f(\mathbf{q}_{k}, \mathbf{d}^{(world)}, \mathbf{s}^{(body)})}{\partial q_z} \\
    \end{bmatrix} \cdot f(\ldots) \\
    &=& \begin{bmatrix}
    2d_yq_z-2d_zq_y & -2d_xq_z+2d_zq_x & 2d_xq_y-2d_yq_x \\
    2d_yq_y+2d_zq_z & 2d_xq_y-4d_yq_x+2d_zq_w & 2d_xq_z-2d_yq_w-4d_zq_x \\
    -4d_xq_y+2d_yq_x-2d_zq_w & 2d_xq_x+2d_zq_z & 2d_xq_w+2d_yq_z-4d_zq_y \\
    4d_xq_z+2d_yq_w+2d_zq_x & -2d_xq_w-4d_yq_z+2d_zq_y & 2d_xq_x+2d_yq_y \\
    \end{bmatrix} \cdot f(\ldots)
    \end{array}\end{split}$$

### Estimate $\mathbf{q}_{t}$ using Gradient Descent Algorithm

- From (2),

    $$\begin{split}\begin{array}{rcl}
    \mathbf{q}_{\omega, t} &=& \,\mathbf{q}_{t-1} + \,\dot{\mathbf{q}}_{\omega, t}\Delta t\\
    &=& \,\mathbf{q}_{t-1} + \frac{1}{2}\big(\,\mathbf{q}_{t-1}\mathbf{\,\omega_{t}^{(body)}}\big)\Delta t
    \end{array}\end{split} \tag{7}$$

- From (6), the gradient quaternion,

    $$\begin{split}\begin{array}{rcl}
    \mathbf{q}_{\nabla,t} &=& \mathbf{q}_{t-1}-\mu_t \ \frac{\nabla f}{\|\nabla f\|} \\
    &=& \mathbf{q}_{t-1}-(\alpha \ \|\dot{\mathbf{q}}_{\omega,t}\| \ \Delta t) \ \frac{\nabla f}{\|\nabla f\|}
    \end{array}\end{split}$$

    where, $\dot{\mathbf{q}}_{\omega,t}$ is the physical orientation rate measured by the gyroscopes and $\alpha>1$ is an augmentation of $\mu$ dealing with the noise of accelerometers and magnetometers. When $\alpha$ is very large, $\mu_t$ becomes very large. As a result,

    $$\mathbf{q}_{\nabla, t} \approx -\mu_t\frac{\nabla f}{\|\nabla f\|} \tag{8}$$

- By applying complimentary filter,

    $$\mathbf{q}_t = \gamma_t  \mathbf{q}_{\nabla, t} + (1-\gamma_t) \mathbf{q}_{\omega, t} \tag{9}$$

    where, $\gamma_t$ ranges from 0 to 1.

- The divergence rate of $\mathbf{q}_{\omega, t}$ is $\beta$ and convergence rate of $\mathbf{q}_{\nabla, t}$ is $\frac{\mu_t}{\Delta t}$. An optimal value of $\gamma_t$  ensures that the weighted divergence of $\mathbf{q}_{\omega, t}$ is equal to the weighted convergence of $\mathbf{q}_{\nabla, t}$.

    $$(1-\gamma_t)\beta = \gamma_t\frac{\mu_t}{\Delta t}$$

    Since $\mu_t$ is very large, $\gamma_{t}$ becomes small and $(1 - \gamma_{t}) \approx 1$. Therefore,

    $$\gamma_{t} \approx \frac{\beta\Delta t}{\mu_t} \tag{10}$$

- Since $(1 - \gamma_{t}) \approx 1$, (9) reduced into,

    $$\mathbf{q}_t = \gamma_t  \mathbf{q}_{\nabla, t} + \mathbf{q}_{\omega, t}$$

    From (7), (8) and (10),

    $$
    \mathbf{q}_t = \mathbf{q}_{t-1} + \big( \dot{\mathbf{q}}_{\omega, t} - \beta\frac{\nabla f}{\|\nabla f\|}\big)\Delta t \tag{11}$$
    
### Orientation from IMU

### Orientation from MARG

## Estimation using Mahony Filter

## Additional Resources

- [AHRS - Attitude from angular rate](https://ahrs.readthedocs.io/en/latest/filters/angular.html)
- [AHRS - Madgwick Orientation Filter](https://ahrs.readthedocs.io/en/latest/filters/madgwick.html)
- [Stanford EE267 - Course Notes on IMU](assets/ee267_notes_imu.pdf)
- [VIRTUAL REALITY By Steven M. LaValle - Chapter 9.1 and 9.2](assets/vrch9.pdf)